In [1]:
import boto3
from astropy.io import fits
from io import BytesIO
import sys
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt
from quickbin import bin2d
import pyarrow.parquet as parquet


In [12]:
data = np.fromfile("/home/bekah/Downloads/e00581-nd-hmask.bin", dtype=bool)
width, height = (800, 800)  # Set your dimensions
array_reshaped = data.reshape((height, width))
img = Image.fromarray(array_reshaped.astype(bool))

In [7]:
from PIL import Image


In [14]:
img.save('mask.png')

In [10]:
existing_fuv = '~/glcat/masks/every3_nuv.csv'
fuv_objs = pd.read_csv(existing_fuv)
fuv_objs = fuv_objs['eclipse'][0:200]

In [11]:
def run_test(eclipse): 
    
    i = str(e).zfill(5)
    key = f"e{i}-nd/e{i}-nd-b00.parquet" 
    
    try: 
        nf = download_photonlist(key, eclipse)
        expt = nf.iloc[len(nf)-1]['t']-nf.iloc[0]['t'] # rough approx not accounting for dead time 

0       580
1       584
2       587
3       754
4       760
       ... 
325    2883
326    2891
327    2898
328    2906
329    3016
Name: eclipse, Length: 330, dtype: int64

In [4]:
bucket_name = 'backplanetest'
s3 = boto3.client('s3')

objects = s3.list_objects_v2(Bucket=bucket_name)

if 'Contents' in objects:
    keys = [obj['Key'] for obj in objects['Contents']]
else:
    print("No objects found in the bucket.")

In [6]:
filtered_keys = []

# Paginate through all the objects in the bucket
paginator = s3.get_paginator('list_objects_v2')
pages = paginator.paginate(Bucket=bucket_name)

# Filter keys based on the suffix
for page in pages:
    if 'Contents' in page:
        for obj in page['Contents']:
            key = obj['Key']
            if key.endswith('-nd-b00.parquet'):
                filtered_keys.append(key)


In [10]:
filtered_keys

['e00581/e00581-nd-b00.parquet',
 'e00582/e00582-nd-b00.parquet',
 'e00584-nd-histmod/e00584-nd-b00.parquet',
 'e00584-nd/e00584-nd-b00.parquet',
 'e00584/e00584-nd-b00.parquet',
 'e00585/e00585-nd-b00.parquet',
 'e00586/e00586-nd-b00.parquet',
 'e00587-nd-histmod/e00587-nd-b00.parquet',
 'e00587-nd/e00587-nd-b00.parquet',
 'e00588/e00588-nd-b00.parquet',
 'e00589/e00589-nd-b00.parquet',
 'e00754-nd-histmod/e00754-nd-b00.parquet',
 'e00754-nd/e00754-nd-b00.parquet',
 'e00759/e00759-nd-b00.parquet',
 'e00760-nd-histmod/e00760-nd-b00.parquet',
 'e00760-nd/e00760-nd-b00.parquet',
 'e00760/e00760-nd-b00.parquet',
 'e00761/e00761-nd-b00.parquet',
 'e00766/e00766-nd-b00.parquet',
 'e00767-nd-histmod/e00767-nd-b00.parquet',
 'e00767-nd/e00767-nd-b00.parquet',
 'e00767/e00767-nd-b00.parquet',
 'e00769/e00769-nd-b00.parquet',
 'e00773/e00773-nd-b00.parquet',
 'e00775-nd-histmod/e00775-nd-b00.parquet',
 'e00775-nd/e00775-nd-b00.parquet',
 'e00775/e00775-nd-b00.parquet',
 'e00780/e00780-nd-b00.pa

In [16]:
df = pd.DataFrame(data={"parq_keys": unique_strings})
df.to_csv("photonfile_keys.csv", sep=',',index=False)

In [9]:
filtered_keys.to_csv("parquet_keys.csv")

AttributeError: 'list' object has no attribute 'to_csv'

In [13]:
strings_list = filtered_keys
filtered_list = [s for s in strings_list if '-histmod' not in s and '-WD' not in s]

# Remove strings with identical first 6 characters
unique_strings = []
seen_prefixes = set()

for s in filtered_list:
    prefix = s[:6]
    if prefix not in seen_prefixes:
        unique_strings.append(s)
        seen_prefixes.add(prefix)

# Result
print(unique_strings)

['e00581/e00581-nd-b00.parquet', 'e00582/e00582-nd-b00.parquet', 'e00584-nd/e00584-nd-b00.parquet', 'e00585/e00585-nd-b00.parquet', 'e00586/e00586-nd-b00.parquet', 'e00587-nd/e00587-nd-b00.parquet', 'e00588/e00588-nd-b00.parquet', 'e00589/e00589-nd-b00.parquet', 'e00754-nd/e00754-nd-b00.parquet', 'e00759/e00759-nd-b00.parquet', 'e00760-nd/e00760-nd-b00.parquet', 'e00761/e00761-nd-b00.parquet', 'e00766/e00766-nd-b00.parquet', 'e00767-nd/e00767-nd-b00.parquet', 'e00769/e00769-nd-b00.parquet', 'e00773/e00773-nd-b00.parquet', 'e00775-nd/e00775-nd-b00.parquet', 'e00780/e00780-nd-b00.parquet', 'e00781/e00781-nd-b00.parquet', 'e00784-nd/e00784-nd-b00.parquet', 'e00788/e00788-nd-b00.parquet', 'e00789/e00789-nd-b00.parquet', 'e00798-nd/e00798-nd-b00.parquet', 'e00800/e00800-nd-b00.parquet', 'e00806/e00806-nd-b00.parquet', 'e00807-nd/e00807-nd-b00.parquet', 'e00812/e00812-nd-b00.parquet', 'e00813/e00813-nd-b00.parquet', 'e00814-nd/e00814-nd-b00.parquet', 'e00815/e00815-nd-b00.parquet', 'e00820/e

In [20]:
filtered_keys[1000][0:6]

'e05096'

In [14]:
# download photonlist 

def download_photonlist(key, ecl):
    s3 = boto3.client('s3')
    try:
        response = s3.get_object(Bucket='backplanetest', Key=key)    
        buffer = BytesIO(response['Body'].read())
        nf = parquet.read_table(buffer, columns=['col', 'row', 'ra', 'dec', 'response','t']).to_pandas()
        return nf 
    except Exception as e:
        if 'NoSuchKey' in str(e):
            print(f"Key '{key}' does not exist. Skipping...")
        else:
            print("Something else went wrong")
            print(str(e))
    return 

In [ ]:
def mask_eclipse(parquet_path, expt):
    
    nf = parquet.read_table(parquet_path,
                           columns=['col', 'row', 'ra', 'dec', 'response']).to_pandas()

    # filter out off detector photons 
    nf['row_rnd'] = nf['row'].round().astype(int)
    nf['col_rnd'] = nf['col'].round().astype(int)

    nf = nf[(nf['col_rnd']<800)&(nf['row_rnd']<800) 
            & (nf['ra']!= 0) & (nf['dec']!= 0) 
            & (nf['col_rnd']>=-50)&(nf['row_rnd']>=-50)]
    mask = pd.notna(nf['ra'])
    nf = nf[mask]

    nbins = 800

    ra_stdev = bin2d(col, row, ra, 'std', nbins)
    dec_stdev =  bin2d(col, row, dec, 'std', nbins)

    response =  bin2d(col, row, response, 'sum', nbins)

    count = bin2d(col, row, ra, 'count', nbins)
    count = count / expt 
    
    # density mask
    density_mask = hist > 400 # not doing low density bc we have response, otherwise gets mixed with background

    # dispersion mask
    disp_mask = ra_dec_disp > .014

    # response map 
    response_mask = response_map > .7 

    # combo mask
    result_array = np.zeros(hist.shape, dtype=int)+1

    result_array[density_mask & disp_mask] = 0
    result_array[response_mask] = 0

    return result_array 

In [ ]:
for e in fuv_objs:
    i = str(e).zfill(5)
    key = f"e{i}-nd/e{i}-nd-b00.parquet" 
    download_photonlist(key, i)

In [15]:
e = 767
i = str(e).zfill(5)
key = f"e{i}-nd/e{i}-nd-b00.parquet" 
nf = download_photonlist(key, i)

In [16]:
nf

,col,row,ra,dec,response
0,545.166992,324.992584,NaN,NaN,0.892357
1,432.622498,211.375626,NaN,NaN,0.865810
2,91.249741,538.526794,NaN,NaN,1.014195
3,475.631622,464.102509,NaN,NaN,0.869943
4,332.834625,182.731247,NaN,NaN,0.865116
...,...,...,...,...,...
13144340,124.963669,551.843140,NaN,NaN,1.028960
13144341,640.749634,526.865051,NaN,NaN,0.986274
13144342,504.397980,615.491455,NaN,NaN,0.964080
13144343,564.666687,720.192810,NaN,NaN,0.891154


In [ ]:
# rought script to generate 200 masks 

# actually I think hotspots and darkspots should be cataloged separately 
existing_fuv = '~/glcat/masks/every3_nuv.csv'
fuv_objs = pd.read_csv(existing_fuv)
fuv_objs = fuv_objs['eclipse'][0:200]

mask_db = np.zeros((800, 800, 200), dtype=bool)

band = "n"

for ind, e in enumerate(fuv_objs): 
    i = str(e).zfill(5)
    key = f"e{i}-nd/e{i}-nd-b00.parquet" 

    nf = parquet.read_table(buffer, columns=['col', 'row', 'ra', 'dec', 'response','t']).to_pandas()
    nf['row_rnd'] = nf['row'].round().astype(int)
    nf['col_rnd'] = nf['col'].round().astype(int)

    nf = nf[(nf['col_rnd']<800)&(nf['row_rnd']<800) 
            & (nf['ra']!= 0) & (nf['dec']!= 0) 
            & (nf['col_rnd']>=-50)&(nf['row_rnd']>=-50)]
    mask = pd.notna(nf['ra'])
    nf = nf[mask]

    expt = nf.iloc[len(nf)-1]['t']-nf.iloc[0]['t'] # rough approx not accounting for dead time

    ra_stdev = bin2d(col, row, ra, 'std', nbins)
    dec_stdev =  bin2d(col, row, dec, 'std', nbins)

    response =  bin2d(col, row, response, 'sum', nbins)

    count = bin2d(col, row, ra, 'count', nbins)
    count = count / expt 
    
    # density mask
    density_mask = count > .5 # not doing low density bc we have response, otherwise gets mixed with background

    # dispersion mask
    disp_mask = ra_dec_disp > .014

    # response map 
    dark_mask = count <= .01 

    # combo mask
    result_array = np.zeros(count.shape, dtype=int)+1
    result_array = np.zeros(count.shape, dtype=int)+1

    # saving hotspot mask per eclipse and band 
    hmask[density_mask & disp_mask] = 0
    hmask.tofile(f'e{i}-{band}d-hmask.bin')
    
    # saving coldspot mask per eclipse and band 
    cmask[dark_mask] = 0
    cmask.tofile(f'e{i}-{band}d-cmask.bin')
